In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, skew, kurtosis

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV
from scipy import stats

import copy as copy

In [2]:
# read the data 
maindir = '/kaggle/input'
filepath_lst = []

for dirname, _, filenames in os.walk(maindir):
    for filename in filenames:
        filepath_lst.append(os.path.join(dirname, filename))

submission = pd.read_csv(filepath_lst[0])
train = pd.read_csv(filepath_lst[1])
test = pd.read_csv(filepath_lst[2])


# EDA

In [3]:
submission.shape, train.shape, test.shape

((67842, 2), (101763, 23), (67842, 22))

In [4]:
train.columns

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')

In [5]:
train.dtypes

id                     int64
loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                    float64
d                    float64
i                    float64
e                    float64
b                    float64
t                    float64
lOCode                 int64
lOComment              int64
lOBlank                int64
locCodeAndComment      int64
uniq_Op              float64
uniq_Opnd            float64
total_Op             float64
total_Opnd           float64
branchCount          float64
defects                 bool
dtype: object

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [7]:
features = train.drop(['defects','id'], axis = 1)
target = train['defects']

In [8]:
float_features = features.select_dtypes(include = 'float').columns.tolist()
int_features = features.select_dtypes(include = 'int').columns.tolist()
print(float_features)
print(int_features)

['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']
['lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment']


In [9]:
features.describe().round().T

,count,mean,std,min,25%,50%,75%,max
loc,101763.0,37.0,55.0,1.0,13.0,22.0,42.0,3442.0
v(g),101763.0,5.0,8.0,1.0,2.0,3.0,6.0,404.0
ev(g),101763.0,3.0,5.0,1.0,1.0,1.0,3.0,165.0
iv(g),101763.0,3.0,6.0,1.0,1.0,2.0,4.0,402.0
n,101763.0,97.0,171.0,0.0,25.0,51.0,111.0,8441.0
v,101763.0,538.0,1271.0,0.0,98.0,233.0,560.0,80843.0
l,101763.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
d,101763.0,14.0,14.0,0.0,6.0,10.0,18.0,418.0
i,101763.0,28.0,23.0,0.0,16.0,23.0,34.0,570.0
e,101763.0,20854.0,190571.0,0.0,565.0,2256.0,10193.0,16846621.0


In [10]:
features.columns.tolist()[0]

'loc'

- outliers, feature scaling

In [11]:
# # check feature distribution
# fig, axes = plt.subplots(3,6, figsize = (15,6), constrained_layout = True)
# axes = axes.flatten()

# for i,j in zip(float_features,axes):
    
#     sns.kdeplot(train, x=i, ax = j, hue = "defects")
#     (mu, sigma) = norm.fit(train[i])
#     xmin, xmax = j.get_xlim()[0], j.get_xlim()[1]
#     x = np.linspace(xmin, xmax, 100)
#     p = norm.pdf(x, mu, sigma)
#     j.plot(x, p, 'k')

- what the distribution plots tell us:

1) number of false cases > number of true classes -> class imbalance -> check distribution after solving the class imbalance issue.

2) data distribution is close to normal distribution, but with higher peaks. 

3) feature values are at similar scales. feature scaling might not be necessary or helpful.

4) outliers. 

In [12]:
# # check for class imbalance
# train.groupby('defects')['id'].count()/train.shape[0]*100

In [13]:
# # check feature correlations with target
# corr = features.corrwith(target).to_frame()
# corr['abs_corr'] = corr.abs()
# corr.rename(columns = {0:'corr'}, inplace = True)
# corr.sort_values(by = 'abs_corr', ascending = 0)

In [14]:
# feature_corr = features.corr().abs()
# feature_corr_v2 = feature_corr.unstack().sort_values(ascending = False).to_frame().reset_index()
# feature_corr_v2 = feature_corr_v2[feature_corr_v2['level_0']!=feature_corr_v2['level_1']]
# feature_corr_v2

In [15]:
# feature_corr_mask = np.triu(feature_corr,k=0)

# plt.figure(figsize=(16,8))
# sns.heatmap(feature_corr,
#            mask = feature_corr_mask,
#            annot = True,
#            fmt = '.1f',
#            cmap = 'coolwarm',
#            linewidth = 0.1,
#            cbar = True
#            )

# plt.suptitle("Features Correlation Heatmap", weight = 'bold', size = 20)
# plt.tight_layout()

- there are several pairs of features that are highly correlated with each other; need to either remove, or use dimension deduction techniques to condense information, or choose tree-method which is immune to feature correlation.

- variables that are highly-correlated with other features: loc, n, v, d, b, IOCode, uniq_Opnd, total_Op, total_Opnd

1) loc & v(g); v(g) & ev(g); n & v
2) loc & iv(g); v(g) & iv(g); n & d
3) loc & n; v(g) & branchCount; n & i
4) loc & v; n & b;
5) loc & b; n & IOCode
6) loc & IOCode; n & IOBlank
7) loc & IOBlank; n & Uniq_Opnd
8) loc & total_Op; n & total_Op;
9) loc & total_Opnd; n & total_Opnd
10) loc & branchCount..

In [16]:
# high_corr_features = feature_corr_v2[feature_corr_v2[0]>0.8]
# high_corr_features.head(3)

In [17]:
# high_corr_features.reset_index().loc[0,'level_0']

In [18]:
# # examine if outliers are the reasons why features are highly-correlated with each other

# fig, axes = plt.subplots(4,6, figsize = (16,6))
# axes = axes.flatten()

# i = 0
# for row in range(0,48,2):
#     feature_a = high_corr_features.reset_index(drop = True).loc[row, ['level_0','level_1']][0]
#     feature_b = high_corr_features.reset_index(drop = True).loc[row, ['level_0','level_1']][1]
    
#     sns.regplot(features, x = feature_a, y = feature_b, ci = None, order = 1, ax = axes[i], scatter_kws={'color':'red', 's':1.5}, line_kws={'color':'black', 'linewidth':1.5})
    
#     i+=1

# plt.tight_layout()
    

In [19]:
# # regplots after deweighting outliers
# fig, axes = plt.subplots(4, 6, figsize = (16, 8))
# axes = axes.flatten()

# i = 0 
# for row in range(0,48,2):
#     feature_a = high_corr_features.reset_index(drop=True).loc[row, 'level_0']
#     feature_b = high_corr_features.reset_index(drop=True).loc[row, 'level_1']
    
#     sns.regplot(features, x = feature_a, y = feature_b, order = 1, robust = True, scatter_kws={'color':'red', 's':1.5}, line_kws={'color':'black', 'linewidth':1.5})
    
#     i+=1

# plt.tight_layout()

- regplot with robust = True takes reallllllly long time to run...better be worth the time. 
- take too much time to run, give up

# Build Baseline Model

In [20]:
# deal with class imbalance issue 
## upsampling
from sklearn.utils import resample

# pos_df = train[train['defects']==True]
# neg_df = train[train['defects']==False]

# pos_upsampled_df = resample(pos_df, replace = True, n_samples = neg_df.shape[0], random_state = 42, stratify = None)
# train_upsampled = pd.concat([pos_upsampled_df, neg_df], axis = 0)
# features = train_upsampled.drop(['defects','id'], axis = 1)
# target = train_upsampled['defects']


# SMOTE
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(sampling_strategy = 'auto', random_state = 42, k_neighbors = 5)
# features_res, target_res = sm.fit_resample(features, target)

# adjust class weight in logistic regression and assign more weights to minority class
n_samples = features.shape[0]
class_weights = n_samples/(2*np.bincount(target))
class_weights_dict = {
    True:class_weights[1],
    False:class_weights[0]
}
class_weights_dict

{True: 2.2061004162330904, False: 0.6465329927953342}

## Feature Engineering

In [21]:
features.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
       'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
       'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount'],
      dtype='object')

In [22]:
# read from others' notebook
features['loc+lOBlank'] = features['loc'] + features['lOBlank']

In [23]:
# from scipy import stats

# def calculate_biserial_corr(column, target:pd.Series):
#     return stats.pointbiserialr(column, target)

# biserial_corr = features.apply(lambda column: calculate_biserial_corr(column, target), axis = 0, raw = False).T
# biserial_corr.rename(columns = {0:'biserial_corr', 1:'p-value'}, inplace = True)
# biserial_corr['abs_biserial_corr'] = biserial_corr['biserial_corr'].abs()
# biserial_corr = biserial_corr[['abs_biserial_corr','biserial_corr', 'p-value']]
# biserial_corr.sort_values('abs_biserial_corr', axis = 0, ascending = 0)

- all variables are significantly correlated with target variables

In [24]:
# plt.figure(figsize=(20,12))
# sns.pairplot(data = train, vars = float_features, diag_kind='kde',
#             kind = 'scatter', hue = 'defects')
# plt.show()
# plt.tight_layout()

In [25]:
# plt.figure(figsize=(20,12))
# sns.pairplot(data = train, vars = int_features, diag_kind='kde',
#             kind = 'scatter', hue = 'defects')
# plt.show()
# plt.tight_layout()

In [26]:
# # plot the scatter plot for all feature-pairs
# from itertools import combinations

# feature_pairs = list(combinations(features.columns,2))

In [27]:
# len(feature_pairs)

In [28]:
target.unique()

array([False,  True])

In [29]:
# # before scaling & normalization 
# fig, axes = plt.subplots(5,5, figsize=(16,8), constrained_layout = True)
# axes = axes.flatten()

# for i in range(21):
#     sns.boxplot(data = train.drop(['id'],axis = 1), x = features.columns.tolist()[i], ax = axes[i])

In [30]:
# # after scaling & normalization 
# fig, axes = plt.subplots(5,5, figsize=(16,8), constrained_layout = True)
# axes = axes.flatten()

# train_copy = copy.deepcopy(train)
# FEATURES = train_copy.drop(['id','defects'], axis = 1)


# for i in range(FEATURES.shape[1]):
#     FEATURES.iloc[:,i] = stats.boxcox(FEATURES.iloc[:,i]+1e-13)[0]

# # standardization
# MMS = MinMaxScaler()
# MMS.fit(FEATURES)
# FEATURES_scaled = MMS.transform(FEATURES)

# for i in range(21):
#     sns.boxplot(data = FEATURES, x = FEATURES.columns.tolist()[i], ax = axes[i])

In [31]:
# # calculate vif for each feature to evaluate multi-collinearity
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif = pd.DataFrame()
# vif["feature"] = features.columns
# vif["vif"] = [variance_inflation_factor(features.values,i) for i in range(len(features.columns.tolist()))]
# vif = vif.sort_values(by = 'vif', ascending = 0)
# vif

In [32]:
# vif['feature'][0:2].tolist()

In [33]:
# # customize min_max_scaler
# def min_max_scaler(train, test, column):
#     max_value = max(train[column].max(), test[column].max())
#     min_value = min(train[column].min(), test[column].min())
    
#     train[column] = (train[column]-min_value)/(max_value-min_value)
#     test[column] = (test[column]-min_value)/(max_value-min_value)
    
#     return train, test
# train_copy = copy.deepcopy(train)

# for col in features.columns.tolist():
#     train_copy, test = min_max_scaler(train_copy, test, col)

In [34]:
# parameters = {
# 'solver':['liblinear','sag','saga'],    
# 'penalty':['l1','l2'],    
#  'max_iter':[300,400,500], 
# 'C': [1.0, 2.0,3.0, 4.0, 5.0]
# }

# best parameters: C:4.0, solver:saga, penalty:l1, max_iter:400

X = features
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [35]:
# boxcox nomarlization

for i in range(X_train.shape[1]):
    X_train.iloc[:,i] = stats.boxcox(X_train.iloc[:,i]+1e-13)[0]

for i in range(X_test.shape[1]):
    X_test.iloc[:,i] = stats.boxcox(X_test.iloc[:,i]+1e-13)[0]

# standardization
mms = MinMaxScaler()
mms.fit(X_train)
X_train_scaled = mms.transform(X_train)
X_test_scaled = mms.transform(X_test)

In [36]:
# train the mod__class_getitem__el
lg = LogisticRegression(solver='saga', penalty='l1', C = 4.0, max_iter=400, class_weight = 'balanced')
lg.fit(X_train_scaled, y_train)
# clf = GridSearchCV(lg, parameters, scoring = 'roc_auc',verbose = 0)
# clf.fit(X_train_scaled, y_train)
y_pred = lg.predict_proba(X_test_scaled)[:,1]
lg_roc_auc_score = roc_auc_score(y_test, y_pred)
print("roc_auc_score of the baseline logistic model is: {:.4f}". format(lg_roc_auc_score))

roc_auc_score of the baseline logistic model is: 0.7825


In [37]:
pd.DataFrame({'feature_name':X_train.columns,'feature_coef':lg.coef_.round(2).reshape(-1)}).sort_values(by='feature_coef', ascending = 1)

,feature_name,feature_coef
0,loc,-28.50
7,d,-4.88
19,total_Opnd,-3.70
8,i,-2.35
14,lOBlank,-2.34
12,lOCode,-1.97
20,branchCount,-0.77
18,total_Op,-0.48
13,lOComment,-0.00
10,b,0.03


In [38]:
# # confusion matrix: error analysis
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# y_label_pred = lg.predict(X_test_scaled)
# cm = confusion_matrix(y_test, y_label_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = lg.classes_)
# disp.plot()
# plt.show()

- lg tends to predict as false, bc of the class imbalance issue in the dataset.

- v, e, t, n is almost shrinked to zero. make sense bc they either rarely correlates with target, or they are highly correlated with other variables.
- baseline model 0.7764 with l1 penalty.
- e1: perform feature scaling by minmaxscaler, 0.7768.
- e2: perform feature normalizing with boxcox on all features, 0.7646
- e3: gridsearch to find best parameters; elastcnet requires C and l1_ratio, making it hard to add it to the process.takes so long to train. didnt find the best parameters that beat 0.7768.
- e4: e1 + e2 + e3, 0.7802
- e5: remove features with top 2 vif score: 'branchCount', 'v(g)', 0.7802
- e6: class imbalance
    - upsampling: 0.7830
    - SMOTE: 0.7911
    - adjust class weight by class frequencies: 0.7801. model tends to predict true label.
- e7: feature engineering + 'loc+lOSBlank'.0.7825
- refer to that notebook and copy its feature engineering technique for every column:
    - takeaways: 
        - almost like a brutal force method to try every possible combinations of features-pair, feature-trainsformation technique, encoding techniques and look at single variable performance. skills are being able to write iterable functions. 
        - optuna can be used to find optimal ensemble weights

# Submission

In [39]:
test['loc+lOBlank'] = test['loc']+test['lOBlank']
test.drop('id', axis = 1, inplace = True)

# normalization
for i in range(test.shape[1]):
    test.iloc[:,i] = stats.boxcox(test.iloc[:,i]+1e-13)[0]

# standardization
mms_test = MinMaxScaler()
mms_test.fit(test)
test_scaled = mms_test.transform(test)

In [40]:
test_pred = lg.predict_proba(test_scaled)[:,1]
test_pred

array([0.4933975 , 0.42595291, 0.89815415, ..., 0.46026498, 0.27924572,
       0.90846635])

In [41]:
submission['defects'] = test_pred
submission

,id,defects
0,101763,0.493398
1,101764,0.425953
2,101765,0.898154
3,101766,0.761986
4,101767,0.436124
...,...,...
67837,169600,0.560001
67838,169601,0.396685
67839,169602,0.460265
67840,169603,0.279246


In [42]:
submission.to_csv("submission.csv", index=False)